In [3]:
import os
import pandas as pd
import googleapiclient.discovery
import re
import time

# API Key 설정
API_KEY = "" 
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# 유튜브 클라이언트 초기화
youtube = googleapiclient.discovery.build(
    YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY
)

# 한글 확인 함수
def contains_korean(text):
    """
    텍스트에 한글이 포함되어 있는지 확인
    """
    return bool(re.search(r"[가-힣]", text))

# 동영상 길이 확인 함수
def is_short_video(duration):
    """
    ISO 8601 형식의 동영상 길이를 분석하여 Shorts (60초 이하)인지 확인
    """
    match = re.match(r"PT(\d+M)?(\d+S)?", duration)
    if match:
        minutes = int(match.group(1)[:-1]) if match.group(1) else 0
        seconds = int(match.group(2)[:-1]) if match.group(2) else 0
        return (minutes * 60 + seconds) <= 60
    return False

# Shorts 동영상 검색 (한국 지역)
def search_shorts_videos(region_code="KR", video_category_id=None, max_results=100):
    """
    제목에 한글이 포함된 한국 숏츠 동영상 검색
    """
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        try:
            request = youtube.videos().list(
                part="snippet,contentDetails",
                chart="mostPopular",
                regionCode=region_code,  # 한국 지역 설정
                videoCategoryId=video_category_id,  # 특정 카테고리 (예: 음악, 엔터테인먼트)
                maxResults=50,  # 요청당 최대 50개
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                title = item["snippet"]["title"]
                duration = item["contentDetails"]["duration"]  # 동영상 길이
                if contains_korean(title) and is_short_video(duration):
                    video_id = item["id"]
                    videos.append({
                        "video_id": video_id,
                        "video_title": title,
                    })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break
        except Exception as e:
            print(f"Shorts 동영상 검색 중 오류 발생: {e}")
            break

    return videos[:max_results]

# 댓글 수집
def get_video_comments(video_id, video_title, max_comments=1000):
    """
    특정 유튜브 동영상의 댓글 가져오기
    """
    comments = []
    next_page_token = None

    while len(comments) < max_comments:
        try:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
                maxResults=100,
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                comment = item["snippet"]["topLevelComment"]["snippet"]
                comment_id = item["id"]
                parent_id = None

                # Top-level comment 추가
                comments.append({
                    "comment_id": comment_id,
                    "parent_id": parent_id,
                    "author": comment["authorDisplayName"],
                    "text": comment["textDisplay"],
                    "likes": comment["likeCount"],
                    "is_reply": False,
                    "video_id": video_id,
                    "video_title": video_title
                })

                # Reply 추가
                for reply in item.get("replies", {}).get("comments", []):
                    comments.append({
                        "comment_id": reply["id"],
                        "parent_id": comment_id,
                        "author": reply["snippet"]["authorDisplayName"],
                        "text": reply["snippet"]["textDisplay"],
                        "likes": reply["snippet"]["likeCount"],
                        "is_reply": True,
                        "video_id": video_id,
                        "video_title": video_title
                    })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break
        except Exception as e:
            print(f"댓글 가져오기 중 오류 발생(video_id: {video_id}): {e}")
            break

    return comments[:max_comments]

# 동영상 및 댓글 저장
def analyze_and_save_comments(videos, output_dir="shorts"):
    """
    동영상 및 댓글 데이터를 수집하고 저장
    """
    os.makedirs(output_dir, exist_ok=True)
    all_comments = []

    for video in videos:
        video_id = video["video_id"]
        video_title = video["video_title"]

        print(f"댓글 가져오는 중: {video_title} ({video_id})")
        comments = get_video_comments(video_id, video_title)
        all_comments.extend(comments)
        time.sleep(1)  # API 요청 제한 보호

    # Save all comments to a CSV file
    if all_comments:
        comments_df = pd.DataFrame(all_comments)
        output_path = os.path.join(output_dir, "comments_shorts.csv")
        comments_df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"댓글 저장 완료: {output_path}")
    else:
        print("댓글이 없습니다.")

# 전체 처리
def process_korean_shorts_videos():
    """
    제목에 한글이 포함된 한국 숏츠 동영상 댓글 수집 및 저장
    """
    output_dir = "shorts"

    # Shorts 동영상 검색 (카테고리 ID: 예시로 엔터테인먼트 ID "24" 사용)
    videos = search_shorts_videos(region_code="KR", video_category_id="24", max_results=100)

    # 댓글 크롤링 및 저장
    analyze_and_save_comments(videos, output_dir)

# 실행
process_korean_shorts_videos()


댓글 가져오는 중: 역시 이효리ㅋㅋ이상순이 나갈 때 하는 말이라는데 #이효리 #이상순 (fIFP2Xd4KaE)
댓글 가져오는 중: 송지효가 12시간 자야되는 이유 (QJRK9zPpNdk)
댓글 가져오는 중: 빨강삼촌 가게 찾아가면 생기는 일 (KArWJqZ9dyw)
댓글 가져오는 중: 유부남들께 꿀팁 전수드립니다 (_k2LsISVhA8)
댓글 가져오는 중: [EN] 차오른 흥을 주체 못한 사파의 밤 & 끝나지 않는 수다 | 풍향고 EP.4 베트남 사파 #유재석 #황정민 #지석진 #양세찬 (YC06Q9XYlQ4)
댓글 가져오는 중: 죽기 전 최고의 선물을 준 누나 #위대한소원 (_DbmHnB9qbo)
댓글 가져오는 중: 거짓말 진짜 못하는 최강록ㅋㅋㅋㅋ (oGfge6SkieU)
댓글 가져오는 중: 심리분석관에게 제대로 긁힌 하정우 #추격자 (vyaJrmzAoy0)
댓글 가져오는 중: 나라 지키는 군인한테 함부로 하면 안되는 이유 ㅋㅋ #푸른거탑 #군대 (GJuemF-alOo)
댓글 가져오는 중: 동종업계 사람들만 아는 무서운 타투 (YcKoCCjvoew)
댓글 가져오는 중: 한국인들의 ‘느긋함’이 바꾼 영화 제목 ㅋㅋ (2PFIU-Nbn9o)
댓글 가져오는 중: 언제나 나를 당당하게 소개 하는 여자#이재곧죽습니다 (07YNhgctyHw)
댓글 가져오는 중: 52살 박진영한테 흰머리가 안 나는 이유ㄷㄷ #shorts #아는형님 (Ots9cxe2nx0)
댓글 가져오는 중: 천사 기영이와 악마 기영이의 싸움#검정고무신 (DkcU31o3mk4)
댓글 가져오는 중: 남돌과 여돌이 뻘쭘한 상황에 대처하는 법 #shorts #아이돌 (oIL1ih6C0nU)
댓글 가져오는 중: 전 연인에게 가장 큰 복수는 (lDHv2OXtzoA)
댓글 가져오는 중: 남동생이 게이라는 결정적 증거ㅋㅋㅋㅋㅋㅋ (j9_zAUH275w)
댓글 가져오는 중: 초대권 발급하지 않는 레스토랑에서 공짜로 식사하는 법 #건방진재벌 #초고속결혼후열애중 #숏차 #드라마 #drama #kdrama #k드라마  #

In [5]:
import os
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 스팸 패턴 정의
SPAM_PATTERNS = [
    r"(클릭|이름|프로필|보러가기|링크|구독|프사|채널)",
    r"9금",  # "I9" 또는 "19" 관련
]

def is_korean(text):
    """
    텍스트에 한글이 포함되어 있는지 확인
    """
    return bool(re.search(r"[가-힣]", text))

def find_similar_comments(comments, threshold=0.8):
    """
    댓글 간 유사도 분석
    """
    if len(comments) < 2:
        return []

    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(comments)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    similar_pairs = []
    n_comments = len(comments)

    for i in range(n_comments):
        for j in range(i + 1, n_comments):
            if similarity_matrix[i, j] >= threshold:
                similar_pairs.append((i, j))
    return similar_pairs

def detect_spam(comments_df):
    """
    스팸 댓글 탐지
    """
    # 대댓글 개수 계산
    comments_df["reply_count"] = comments_df.groupby("parent_id")["parent_id"].transform("count")
    comments_df["reply_count"] = comments_df["reply_count"].fillna(0)

    # 유사 댓글 및 스팸 패턴 분석
    similar_comments = find_similar_comments(comments_df["text"].astype(str).tolist())
    spam_indices = set()

    for i, j in similar_comments:
        if comments_df.iloc[j]["is_reply"]:
            comment_text = comments_df.iloc[j]["text"]
            if is_korean(comment_text) and comments_df.iloc[j]["reply_count"] >= 2:
                for pattern in SPAM_PATTERNS:
                    if re.search(pattern, comment_text):
                        spam_indices.add(j)

    comments_df["is_spam"] = comments_df.index.isin(spam_indices).astype(int)
    return comments_df

def classify_and_save_spam(input_file="shorts/comments_shorts.csv", output_file="shorts/comments_shorts_spam.csv"):
    """
    저장된 단일 댓글 데이터를 스팸 여부로 분류하여 저장
    """
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    try:
        comments_df = pd.read_csv(input_file)

        if not comments_df.empty:
            # 스팸 탐지
            classified_df = detect_spam(comments_df)
            
            # 스팸만 필터링
            spam_only_df = classified_df[classified_df["is_spam"] == 1]
            
            # 스팸 댓글 저장
            spam_only_df.to_csv(output_file, index=False, encoding="utf-8-sig")
            print(f"스팸 댓글만 저장 완료: {output_file}")
        else:
            print(f"빈 파일입니다: {input_file}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {input_file} - {e}")

# 실행
classify_and_save_spam()


스팸 댓글만 저장 완료: shorts_data_korean/comments/comments_spam.csv
